In [1]:
import ee, os
import geemap
Map = geemap.Map(center=[30, 113], zoom=7) 
Map

Map(center=[30, 113], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…

In [2]:
h = 110
v = 25
#Year
IniYear = 2020

LowLat = v
UpLat = v+5
LeftLon = h
RightLon = h+5

#Regions
region = ee.Geometry.Rectangle(LeftLon,LowLat,RightLon,UpLat);
Map.addLayer(region)

dem = ee.Image('USGS/SRTMGL1_003')
#Set region and starting and ending time
start_date = str(IniYear) + '-01-01';
end_date = str(IniYear) + '-12-31';
print(start_date)
print(end_date)

2020-01-01
2020-12-31


In [3]:
collection8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
    .filterBounds(region)\
    .filterDate(start_date, end_date)\
    .select( #Rename LC8 bands same as L5/7
      ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']
      ,['B1', 'B2', 'B3', 'B4', 'B5', 'B7'])
print('Landsat8')
# print(collection8.size().getInfo())

Landsat8


In [4]:
Map.addLayer(collection8.mosaic().toInt16().clip(region), {}, 'image')
Map

Map(bottom=13819.0, center=[30, 113], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=H…

In [5]:
image = collection8.mosaic().toInt16().clip(region)

In [15]:
from user_gee import *
outfile = 'E:/Work/TEMP'
if __name__ == '__main__':
    clip_dow_merge(region, image, outfile, 200, 2)

总耗时:00h00m10s


In [ ]:
import rasterio
from rasterio.merge import merge
import shutil
from glob import glob

files = glob(path + "/*.tif")
src_files_to_mosaic = []
for tif_f in files:
    src = rasterio.open(tif_f)
    src_files_to_mosaic.append(src)
mosaic, out_trans = merge(src_files_to_mosaic)
out_meta = src.meta.copy()
out_meta.update({"driver": "GTiff",
                 "height": mosaic.shape[1],
                 "width": mosaic.shape[2],
                 "transform": out_trans,
                 })
with rasterio.open(outfile + ".tif", "w", **out_meta) as dest:
    dest.write(mosaic)
for src in src_files_to_mosaic:
    src.close()
shutil.rmtree(path)

In [7]:
import rasterio
from rasterio.merge import merge
import shutil
from glob import glob

outfile = 'E:/Work/TEMP'
path = outfile + '_mk'
files = glob(path + "/*.tif")
src_files_to_mosaic = []
for tif_f in files:
    src = rasterio.open(tif_f)
    src_files_to_mosaic.append(src)
mosaic, out_trans = merge(src_files_to_mosaic)


In [8]:
out_meta = src.meta.copy()
out_meta.update({"driver": "GTiff",
                 "height": mosaic.shape[1],
                 "width": mosaic.shape[2],
                 "transform": out_trans,
                 })

In [9]:
with rasterio.open(outfile + ".tif", "w", **out_meta) as dest:
    dest.write(mosaic)

In [25]:
bounds = region.bounds(maxError=0.1, proj=ee.Projection('EPSG:3857')).getInfo()

In [51]:
mosaic.shape

(6, 3155, 2784)

In [35]:
import numpy as np
poy = np.array(bounds['coordinates'][0])
min_x = poy[:, 0].min()
max_x = poy[:, 0].max()
min_y = poy[:, 1].min()
max_y = poy[:, 1].max()

In [36]:
print(min_x, max_x, min_y, max_y)

12245143.987260092 12801741.44122646 2875744.6243522423 3506587.6599170286


In [63]:
outfile = 'E:/Work/TEMP1'
scale = 200
height = int(max_y / slace) - int(min_y / slace) + 1
width = int(max_x / slace) - int(min_x / slace) + 1
transform = [scale, 0, int(min_x / scale) * scale , 0, -scale, (int(max_y / scale) + 1) * scale]

In [65]:
out_meta = src.meta.copy()
out_meta.update({"driver": "GTiff",
                 "height": height,
                 "width": width,
                 "transform": transform,
                 })

In [50]:
with rasterio.open(outfile + ".tif", "w", **out_meta) as dest:
    for tif_f in files:
        with rasterio.open(tif_f) as src:
            
    

[12543200.0, 200.0, 0.0, 3506600.0, 0.0, -200.0]

In [66]:
rasterio.windows()

Affine(200.0, 0.0, 12543200.0,
       0.0, -200.0, 3506600.0)

In [64]:
transform

[200, 0, 12245000, 0, -200, 3506600]